# Afinidad de Términos en Twitter

## Cargar el archivo
Se extrajeron tweets del año 2016 en los que se menciona al BBVA de la solución GNIP

Instalamos las librerías necesarias para leer archivos de Excel

In [ ]:
pip install openpyxl

In [ ]:
pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
from sklearn.feature_extraction.text import CountVectorizer
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import time

In [ ]:
df = pd.read_excel('../data/Tweets_BBVA.xlsx', index_col="id")
df.head()

Extraemos el campo "body" que contiene el tweet

In [ ]:
tweet = df["body"]
tweet.head()

#### Creamos la Matriz Documento-Término (DTM)

In [ ]:
vec = CountVectorizer()
X = vec.fit_transform(tweet)

In [ ]:
dtm = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
dtm.head()

In [ ]:
dtm.info()

#### Removemos las columnas de las stopwords y de los términos poco frecuentes

In [ ]:
column = pd.DataFrame(dtm.sum())
column.head()

In [ ]:
column[column[0]>300].index

In [ ]:
dtm.drop(columns=['al', 'co', 'con', 'de', 'del', 'el', 'en', 'es', 'https', 'la', 'las', 'lo', 'los', 'me', 'mi', 'rt',
                  'no', 'para', 'por', 'que', 'se', 'su', 'un', 'una', 'at', 'in', 'ya', 'sus', 'nos'], inplace=True)

In [ ]:
stop = list(column[column[0]<5].index)

In [ ]:
dtm.drop(columns=stop, inplace=True)

#### Realizamos un Encoding antes de utilizar los algoritmos de Afinidad

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
dtm_nw = dtm.applymap(encode_units)
dtm_nw.head()

#### Seleccionamos reglas con un mínimo de Soporte

In [ ]:
freq_words = apriori(dtm_nw, min_support=0.02, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
freq_words['freq'] = freq_words['itemsets'].apply(lambda x: len(x))
freq_words.head()

In [ ]:
freq_words.info()

#### Ahora generamos las reglas, ordenando de acuerdo al Lift

In [ ]:
res = association_rules(freq_words, metric='lift').sort_values('lift', ascending=False).reset_index(drop=True)
res.head(20)

In [ ]:
res.info()

#### Ahora podemos transformar el DataFrame en un Grafo Dirigido con el Soporte como peso de la relación entre los términos

In [ ]:
G = nx.from_pandas_edgelist(res, source = 'antecedents', target = 'consequents', create_using=nx.DiGraph(), edge_attr='support')
print(nx.info(G))

Crear la función top_nodes que mostrará los valores más altos de un diccionario

In [ ]:
def get_top_nodes(cdict, num=5):
    top_nodes ={}
    for i in range(num):
        top_nodes =dict(
            sorted(cdict.items(), key=lambda x: x[1], reverse=True)[:num]
            )
        return top_nodes

#### Representación de las relaciones entre Términos

Guardar el grado de cada nodo en un diccionario

In [ ]:
gdeg=G.degree()

In [ ]:
get_top_nodes(dict(gdeg))

In [ ]:
plt.figure(figsize=(32,18))
edges = G.edges()
weights = [G[u][v]['support'] for u,v in edges]
nx.draw_networkx(G, width=weights, node_size=[100*val for(node,val)in gdeg])
plt.show()

Elaborado por Luis Cajachahua bajo licencia MIT (2022)